In [ ]:
from pynq import Overlay
from pynq import allocate
import numpy as np


In [ ]:
ol = Overlay("./design_1.bit")
ol.ip_dict

In [ ]:
dma0 = ol.axi_dma_0
dma0_send = ol.axi_dma_0.sendchannel
dma0_recv = ol.axi_dma_0.recvchannel

dma1 = ol.axi_dma_1
dma1_send = ol.axi_dma_1.sendchannel
dma1_recv = ol.axi_dma_1.recvchannel

hls_ip = ol.axis_fp_example_0
hls_ip.register_map

In [ ]:
CONTROL_REGISTER = 0x0
hls_ip.write(CONTROL_REGISTER, 0x1) # 0x81 will set bit 0
hls_ip.register_map

In [ ]:
NUM_SAMPLES = 20
input_buffer0 = allocate(shape=(NUM_SAMPLES,), dtype=np.float32)
input_buffer1 = allocate(shape=(NUM_SAMPLES,), dtype=np.float32)
output_buffer0 = allocate(shape=(NUM_SAMPLES,), dtype=np.float32)
output_buffer1 = allocate(shape=(NUM_SAMPLES,), dtype=np.float32)

a = [i for i in range(NUM_SAMPLES)]
a1 = np.float32(a)
a2 = a1*2 + 1.0
np.copyto(input_buffer0, a1)
np.copyto(input_buffer1, a2)


In [ ]:
for i in range(NUM_SAMPLES):
    print(input_buffer0[i], input_buffer1[i], output_buffer0[i], output_buffer1[i])

In [ ]:
dma0_recv.transfer(output_buffer0)
dma1_recv.transfer(output_buffer1)
dma0_send.transfer(input_buffer0)
dma1_send.transfer(input_buffer1)
dma0_send.wait()
dma1_send.wait()
dma0_recv.wait()
dma1_recv.wait()

In [ ]:
dma0.register_map

In [ ]:
dma1.register_map

In [ ]:
# out = in1 + in2 + 5
for i in range(NUM_SAMPLES):
    if np.absolute(output_buffer0[i] - input_buffer0[i] - input_buffer1[i]) < 0.001:
        print(input_buffer0[i], '+', input_buffer1[i],  'is roughly', output_buffer0[i])
    else:
        print('error!', input_buffer0[i], '+', input_buffer1[i], 'should not be', output_buffer0[i])
    if np.absolute(output_buffer1[i] - input_buffer0[i] + input_buffer1[i]) < 0.001:
        print(input_buffer0[i], '-', input_buffer1[i],  'is roughly', output_buffer1[i])
    else:
        print('error!', input_buffer0[i], '+', input_buffer1[i], 'should not be', output_buffer1[i])

In [ ]:
del input_buffer0
del input_buffer1
del output_buffer0
del output_buffer1
del ol